In [1]:
import os
import sys
import time

import numpy as np
import pandas as pd
import math
from tqdm import tqdm

from trackml.dataset import load_event
from trackml.randomize import shuffle_hits
from trackml.score import score_event

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns
import collections as coll
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KDTree

sys.path.append("../..")
import merge as merge
import extension as ext
import zroutlier as zro
import free_hits as free
import track_score as score2
import straight_tracks as strt
import eda_utils as eda
import r0outlier as r0o

%matplotlib inline

In [2]:
TRAIN_PATH = '../../../input/train_1'
event_id = 1000

In [3]:

event_prefix = 'event00000' + str(event_id)
hits, cells, particles, truth = load_event(os.path.join(TRAIN_PATH, event_prefix))

mem_bytes = (hits.memory_usage(index=True).sum() 
             + cells.memory_usage(index=True).sum() 
             + particles.memory_usage(index=True).sum() 
             + truth.memory_usage(index=True).sum())
print('{} memory usage {:.2f} MB'.format(event_prefix, mem_bytes / 2**20))

event000001000 memory usage 18.46 MB


In [4]:
helix_root_path = '../../1000_r0_exp3/event_' + str(event_id) + '_labels_train_helix'
labels_helix1 = pd.read_csv(helix_root_path + '1.csv').label.values
labels_helix2 = pd.read_csv(helix_root_path + '2.csv').label.values
labels_helix3 = pd.read_csv(helix_root_path + '3.csv').label.values
labels_helix4 = pd.read_csv(helix_root_path + '4.csv').label.values
labels_helix5 = pd.read_csv(helix_root_path + '5.csv').label.values
labels_helix6 = pd.read_csv(helix_root_path + '6.csv').label.values
labels_helix7 = pd.read_csv(helix_root_path + '7.csv').label.values
labels_helix8 = pd.read_csv(helix_root_path + '8.csv').label.values
labels_helix9 = pd.read_csv(helix_root_path + '9.csv').label.values
labels_helix10 = pd.read_csv(helix_root_path + '10.csv').label.values
labels_helix11 = pd.read_csv(helix_root_path + '11.csv').label.values
labels_helix12 = pd.read_csv(helix_root_path + '12.csv').label.values
labels_helix13 = pd.read_csv(helix_root_path + '13.csv').label.values
labels_helix14 = pd.read_csv(helix_root_path + '14.csv').label.values
labels_helix15 = pd.read_csv(helix_root_path + '15.csv').label.values
labels_helix16 = pd.read_csv(helix_root_path + '16.csv').label.values
labels_helix17 = pd.read_csv(helix_root_path + '17.csv').label.values
labels_helix18 = pd.read_csv(helix_root_path + '18.csv').label.values
labels_helix19 = pd.read_csv(helix_root_path + '19.csv').label.values
labels_helix20 = pd.read_csv(helix_root_path + '20.csv').label.values

In [5]:
def create_one_event_submission(event_id, hits, labels):
    sub_data = np.column_stack(([event_id]*len(hits), hits.hit_id.values, labels))
    submission = pd.DataFrame(data=sub_data, columns=["event_id", "hit_id", "track_id"]).astype(int)
    return submission

def score_one_submission(event_id, hits, labels, truth):
    submission = create_one_event_submission(event_id, hits, labels)
    score = score_event(truth, submission)
    print("Score for event %d: %.8f" % (event_id, score))

In [ ]:
#hits['z_abs'] = hits.z.abs()
#hits['r'] = np.sqrt(hits.x**2+hits.y**2)
#hits['zr'] = hits['z'] / hits['r']
#hits['azr'] = np.arctan2(hits['z'], hits['r'])

In [ ]:
# Run this block if you want to see the distribution of perfect vs short vs imperfect vs horrible
# tracks, by comparing to ground truth
helix6 = np.copy(labels_helix9)
#helix6 = merge.remove_outliers(helix6, hits, smallest_track_size=6, aggressive=False, print_counts=False)
(helix6, small_count) = merge.remove_small_tracks(helix6, smallest_track_size=6)
#helix6 = r0o.remove_badr0_tracks(helix6, hits)
helix6 = merge.renumber_labels(helix6)
tracks = np.unique(helix6)
short_tracks = []
perfect_tracks = []
imperfect_tracks = []
horrible_tracks = []
for track in tracks:
    if track == 0: continue
    tix = np.where(helix6 == track)[0]
    if len(tix) < 6: continue
    else:
        (is_match, correct,incorrect) = eda.track_distance_from_truth(track, helix6, hits, truth)
        if is_match:
            perfect_tracks.append(track)
        elif incorrect == 0:
            short_tracks.append(track)
        elif incorrect <= 4 and correct >= incorrect:
            imperfect_tracks.append(track)
        else:
            horrible_tracks.append(track)

print('Total tracks:     ' + str(len(tracks)))
print('Perfect tracks:   ' + str(len(perfect_tracks)))
print('Short tracks:     ' + str(len(short_tracks)))
print('Imperfect tracks: ' + str(len(imperfect_tracks)))
print('Horrible tracks:  ' + str(len(horrible_tracks)))
# Helix1: 8143, perfect 1972, short 1364, imperfect 3547, horrible 1259
# Helix2: 7666, perfect 2020, short 1170, imperfect 3522, horrible 953
# Helix3: 6717, perfect 1783, short 1589, imperfect 2696, horrible 648
# Helix4: 6688, perfect 1547, short 1649, imperfect 2780, horrible 711
# Helix5: 6645, perfect 1600, short 1763, imperfect 2781, horrible 500
# Helix6: 7436, perfect 1755, short 1716, imperfect 3361, horrible 603
# Helix7: 7451, perfect 1795, short 1795, imperfect 3299, horrible 589
# Helix8: 7421, perfect 1734, short 1638, imperfect 3302, horrible 746
# Helix9: 7414, perfect 1807, short 1699, imperfect 3276, horrible 631

In [6]:
def split_tracks_based_on_quality(labels, hits):
    """Split input tracks into 3 categories - strong, medium, weak.
    Splitting is determined mainly by how consistent the track helix curvature is."""
    strong_tracks = []
    medium_tracks = []
    weak_tracks = []
    hits['z_abs'] = hits.z.abs()
    tracks, counts = np.unique(labels, return_counts=True)
    strong_labels = np.zeros_like(labels)
    medium_labels = np.zeros_like(labels)
    weak_labels = np.zeros_like(labels)
    for ix, track in enumerate(tracks):
        if track == 0: continue
        if counts[ix] < 5:
            if counts[ix] > 3:
                medium_tracks.append(track)
            # else, discard, too short.
            continue
        (curv1, curv2, curv3) = r0o.find_track_curvature(track, labels, hits)
        if np.sign(curv1) != np.sign(curv2) or np.sign(curv1) != np.sign(curv3):
            weak_tracks.append(track)
            continue

        c1 = min(abs(curv1), abs(curv2))
        c2 = max(abs(curv1), abs(curv2))
        c3 = abs(curv3)
        ratio = 1.0 - c1/c2
        if ratio > 0.50:
            weak_tracks.append(track)
        elif ratio < 0.2:
            if counts[ix] > 20 or r0o.is_horrible_track2(track, labels, hits):
                medium_tracks.append(track)
            else:
                strong_tracks.append(track)
        else:
            medium_tracks.append(track)

    for track in strong_tracks:
        strong_labels[labels==track] = track
    for track in medium_tracks:
        medium_labels[labels==track] = track
    for track in weak_tracks:
        weak_labels[labels==track] = track

    return (strong_labels, medium_labels, weak_labels)

In [ ]:
def display_track_quality(labels, hits, truth):
    tracks, counts = np.unique(labels, return_counts=True)
    short_tracks = 0
    perfect_tracks = 0
    imperfect_tracks = 0
    horrible_tracks = 0
    for ix, track in enumerate(tracks):
        if track == 0: continue
        if counts[ix] < 6: continue

        (is_match, correct,incorrect) = eda.track_distance_from_truth(track, labels, hits, truth)
        if is_match:
            perfect_tracks = perfect_tracks + 1
        elif incorrect == 0:
            short_tracks = short_tracks + 1
        elif incorrect <= 4 and correct >= incorrect:
            imperfect_tracks = imperfect_tracks + 1
        else:
            horrible_tracks = horrible_tracks + 1

    print('Total tracks:     ' + str(len(tracks)))
    print('Perfect tracks:   ' + str(perfect_tracks))
    print('Short tracks:     ' + str(short_tracks))
    print('Imperfect tracks: ' + str(imperfect_tracks))
    print('Horrible tracks:  ' + str(horrible_tracks))

In [ ]:
labels = np.copy(labels_helix6)
score_one_submission(event_id, hits, labels, truth) # 0.63580502
(strong, medium, weak) = split_tracks_based_on_quality(labels, hits)
score_one_submission(event_id, hits, strong, truth)
score_one_submission(event_id, hits, medium, truth)
score_one_submission(event_id, hits, weak, truth)
# (0.5 weak, 0.1 strong): 0.28793004 (1776/3737), 0.31400817 (1582/5075), 0.03201160 (28/646)
# (0.5 weak, 0.15 strong): 0.35599925 (2247/4484), 0.24593896 (1111/4328), 0.03201160 (28/646)
# (0.5 weak, 0.2 strong): 0.40582813 (2604/5035), 0.19611008 (754/3777), 0.03201160 (28/646)

In [ ]:
display_track_quality(strong, hits, truth)
display_track_quality(medium, hits, truth)
display_track_quality(weak, hits, truth)

In [7]:
labels_helix1f = merge.remove_outliers(labels_helix1, hits, cells, print_counts=False)
labels_helix2f = merge.remove_outliers(labels_helix2, hits, cells, print_counts=False)
labels_helix3f = merge.remove_outliers(labels_helix3, hits, cells, print_counts=False)
labels_helix4f = merge.remove_outliers(labels_helix4, hits, cells, print_counts=False)
labels_helix5f = merge.remove_outliers(labels_helix5, hits, cells, print_counts=False)
# LIAM: Add more outlier removal here. Explore different cutoffs (> 0.2?)
labels_helix1f = r0o.remove_badr0_tracks(labels_helix1f, hits)
labels_helix2f = r0o.remove_badr0_tracks(labels_helix2f, hits)
labels_helix3f = r0o.remove_badr0_tracks(labels_helix3f, hits)
labels_helix4f = r0o.remove_badr0_tracks(labels_helix4f, hits)
labels_helix5f = r0o.remove_badr0_tracks(labels_helix5f, hits)

In [8]:
all_labels = []
all_labels.append(labels_helix1f)
all_labels.append(labels_helix2f)
all_labels.append(labels_helix6)
all_labels.append(labels_helix7)
all_labels.append(labels_helix8)
all_labels.append(labels_helix9)
all_labels.append(labels_helix10)
all_labels.append(labels_helix11)
all_labels.append(labels_helix12)
all_labels.append(labels_helix13)
all_labels.append(labels_helix14)
all_labels.append(labels_helix15)
all_labels.append(labels_helix16)
all_labels.append(labels_helix17)
all_labels.append(labels_helix18)
all_labels.append(labels_helix19)
all_labels.append(labels_helix20)
all_labels.append(labels_helix5f)
all_labels.append(labels_helix3f)
all_labels.append(labels_helix4f)
strong_labels = []
medium_labels = []
weak_labels = []
for label in all_labels:
    (strong, medium, weak) = split_tracks_based_on_quality(label, hits)
    strong_labels.append(strong)
    medium_labels.append(medium)
    weak_labels.append(weak)

In [9]:
def merge_all_labels(all_labels, hits, truth):
    merge_count = 0
    labels_merged = np.copy(all_labels[0])
    for i in range(len(all_labels)):
        if i == 0: continue
        labels_merged = merge.heuristic_merge_tracks(labels_merged, all_labels[i], hits, overwrite_limit=6, print_summary=False)
        merge_count = merge_count + 1
        #message = 'Merged loop 1-' + str(i+1) + ' score for event '
        #display_score(event_id, hits, labels_merged, truth, message)
        score_one_submission(event_id, hits, labels_merged, truth)
    return labels_merged

In [10]:
all_merged = merge_all_labels(all_labels, hits, truth)
# No outlier removal, order 1-9: 0.70948133
# outlier removal, 1-2,6-9,5,3,4: 0.71563907
# + r0 outlier rem: 0.71652579
# + more models(10-20):

Score for event 1000: 0.69349524
Score for event 1000: 0.71305367
Score for event 1000: 0.71612071
Score for event 1000: 0.71541384
Score for event 1000: 0.71610327
Score for event 1000: 0.71606276
Score for event 1000: 0.71678072
Score for event 1000: 0.71758944
Score for event 1000: 0.71822985
Score for event 1000: 0.71836265
Score for event 1000: 0.71783664
Score for event 1000: 0.71893532
Score for event 1000: 0.71786945
Score for event 1000: 0.71926097
Score for event 1000: 0.71846895
Score for event 1000: 0.71824668
Score for event 1000: 0.71838602
Score for event 1000: 0.71876750
Score for event 1000: 0.71835409


In [11]:
strong_merged = merge_all_labels(strong_labels, hits, truth)
# 0.1 cutoff: 0.47028638
# 0.15 cutoff: 0.54866460
# 0.2 cutoff: 0.59883933
# outlier rem, 0.2 cutoff: 0.60684391
# +r0 out. rem: 0.60669428
# + models 10-20: 0.62883099

Score for event 1000: 0.51293564
Score for event 1000: 0.55977605
Score for event 1000: 0.57273489
Score for event 1000: 0.58420738
Score for event 1000: 0.58764724
Score for event 1000: 0.59721682
Score for event 1000: 0.60144344
Score for event 1000: 0.60531215
Score for event 1000: 0.60768387
Score for event 1000: 0.61053109
Score for event 1000: 0.61255038
Score for event 1000: 0.61346986
Score for event 1000: 0.61420426
Score for event 1000: 0.61569522
Score for event 1000: 0.61588087
Score for event 1000: 0.61652494
Score for event 1000: 0.62397586
Score for event 1000: 0.62753788
Score for event 1000: 0.62883099


In [23]:
m2_lbl = medium_labels[0:13]
m2_lbl.append(medium_labels[-3])
m2_lbl.append(medium_labels[-2])
m2_lbl.append(medium_labels[-1])
medium_merged = merge_all_labels(m2_lbl, hits, truth)
#medium_merged = merge_all_labels(medium_labels, hits, truth)
# 0.1 cutoff: 0.50709876
# 0.15 cutoff: 0.43838218
# 0.2 cutoff: 0.38796599
# outlier rem, 0.2 cutoff: 0.36414178
# +r0 out. rem: 0.36216201
# + models 10-20: 0.45574288
# + models 1,2,6,7,8,9,10,5,3,4: 0.42359094
# + models 1,2,6-15,5,3,4: 0.44986680

Score for event 1000: 0.22760629
Score for event 1000: 0.28729900
Score for event 1000: 0.31137640
Score for event 1000: 0.33276157
Score for event 1000: 0.34104471
Score for event 1000: 0.38566285
Score for event 1000: 0.40904801
Score for event 1000: 0.41738492
Score for event 1000: 0.42570800
Score for event 1000: 0.43047799
Score for event 1000: 0.43516841
Score for event 1000: 0.43874598
Score for event 1000: 0.44446221
Score for event 1000: 0.44761954
Score for event 1000: 0.44986680


In [19]:
w2_lbl = weak_labels[0:8]
w2_lbl.append(weak_labels[-3])
w2_lbl.append(weak_labels[-2])
w2_lbl.append(weak_labels[-1])
weak_merged = merge_all_labels(w2_lbl, hits, truth)
#weak_merged = merge_all_labels(weak_labels, hits, truth)
# 0.5 cutoff: 0.09220529
# 0.5 cutoff + outlier rem: 0.08541034
# + r0 out. rem: 0.08497520
# + models 10-20: 0.11515280
# + models 1,2,6,7,8,9,10,5,3,4: 0.10249580

Score for event 1000: 0.04441391
Score for event 1000: 0.06550528
Score for event 1000: 0.06957069
Score for event 1000: 0.07678826
Score for event 1000: 0.07941898
Score for event 1000: 0.09149716
Score for event 1000: 0.09832986
Score for event 1000: 0.10038455
Score for event 1000: 0.10169605
Score for event 1000: 0.10249580


In [24]:
labels_merged = merge.heuristic_merge_tracks(strong_merged, medium_merged, hits, overwrite_limit=3, print_summary=False)
score_one_submission(event_id, hits, labels_merged, truth)
# strong=0.1 cutoff: 0.7186 (overwrite=3)
# strong=0.15 cutoff: 0.7192 (overwrite=3)
# strong=0.2 cutoff: 0.7196
# outlier rem., 0.2 cutoff: 0.7246
# + r0 out. rem: 0.7260
# + models 10-20: 0.73364939
# only 10 medium models: 0.73336881
# only 15 medium models: 0.73329531

Score for event 1000: 0.73329531


In [25]:
labels_merged2 = merge.heuristic_merge_tracks(labels_merged, weak_merged, hits, overwrite_limit=1, print_summary=False)
score_one_submission(event_id, hits, labels_merged2, truth)
# strong=0.1 cutoff: 0.7228 (overwrite=1)
# strong=0.15 cutoff: 0.7237 (overwrite=1)
# strong=0.2 cutoff: 0.7241 (overwrite=1)
# outlier rem. strong=0.2: 0.7293
# +r0 out. rem: 0.7311
# +models 10-14: 0.735?
# + models 10-20: 0.73648855
# only 10 weak models: 0.73707780
# only 15 medium models: 0.73671680

Score for event 1000: 0.73671680


In [26]:
labels = strt.extend_straight_tracks(labels_merged2, hits)
score_one_submission(event_id, hits, labels, truth)
labels = free.assign_free_hits(labels, hits)
score_one_submission(event_id, hits, labels, truth)
# scores: 0.73736782, 0.73874326
# only 15 medium models: 0.73845154

Score for event 1000: 0.73724001
Score for event 1000: 0.73845154


In [ ]:
labels_helix6f = remove_outliers2(labels_helix6, hits, cells, print_counts=True)
score_one_submission(event_id, hits, labels_helix6, truth)
score_one_submission(event_id, hits, labels_helix6f, truth)

In [ ]:
def remove_track_outliers2(track, labels, hits, cells, aggressive):
    labels = np.copy(labels)
    found_bad_volume = 0
    found_bad_cell = 0
    found_bad_dimension = 0
    found_bad_slope = 0
    found_bad_z = 0
    found_bad_zr = 0

    if True:
        outlier_zr = zro.find_track_outliers_zr(track, labels, hits)
        if len(outlier_zr) > 0:
            #print('track ' + str(track) + ' zr outliers: ' + str(outlier_zr))
            found_bad_zr = found_bad_zr + len(outlier_zr)
            for oix in outlier_zr:
                labels[oix] = 0

    if True:
        # Check if the sorted hits (on z-axis) go through the volumes
        # and layers in the expected order
        duplicatez_ix = merge.find_duplicate_z_using_zr(track, labels, hits)
        if len(duplicatez_ix) > 0:
            #print('track ' + str(track) + ' duplicate z: ' + str(duplicatez_ix))
            found_bad_z = found_bad_z + len(duplicatez_ix)
            for bzix in duplicatez_ix:
                labels[bzix] = 0

    if False:#True:
        # Check the helix slope, discard hits that do not match
        outlier_slope_ix = merge.remove_track_outliers_slope(track, labels, hits)
        if len(outlier_slope_ix) > 0:
            #print('track ' + str(track) + ' slope outliers: ' + str(outlier_slope_ix))
            found_bad_slope = found_bad_slope + len(outlier_slope_ix)
            for oix in outlier_slope_ix:
                labels[oix] = 0

    return (labels, found_bad_volume, found_bad_dimension, found_bad_z, found_bad_slope, found_bad_zr, found_bad_cell)


def remove_outliers2(labels, hits, cells, smallest_track_size=2, aggressive=False, print_counts=True):
    tracks = np.unique(labels)
    hits['z_abs'] = hits.z.abs()
    hits['r'] = np.sqrt(hits.x**2+hits.y**2)
    hits['a0'] = np.arctan2(hits.y,hits.x)
    hits['zr'] = hits['z'] / hits['r']
    count_rem_volume = 0
    count_rem_dimension = 0
    count_duplicatez = 0
    count_rem_slope = 0
    count_small_tracks = 0
    count_zr = 0
    count_cell = 0
    for track in tracks:
        if track == 0:
            continue
        track_hits = np.where(labels == track)[0]
        if len(track_hits) > 3:
            (labels, c1, c2, c3, c4, c5, c6) = remove_track_outliers2(track, labels, hits, cells, aggressive)
            count_rem_volume = count_rem_volume + c1
            count_rem_dimension = count_rem_dimension + c2
            count_duplicatez = count_duplicatez + c3
            count_rem_slope = count_rem_slope + c4
            count_zr = count_zr + c5
            count_cell = count_cell + c6

    # Remove small tracks, we do not get any score for those. This is done
    # last, in case removing the outliers (above) removed enough hits
    # from a track to make them smaller than the threshold.
    (labels, count_small_tracks) = merge.remove_small_tracks(labels, smallest_track_size=smallest_track_size)

    if print_counts:
        print('Total removed due to bad cells: ' + str(count_cell))
        print('Total removed due to bad volumes: ' + str(count_rem_volume))
        print('Total removed due to bad zr values: ' + str(count_zr))
        print('Total removed due to bad dimensions: ' + str(count_rem_dimension))
        print('Total removed due to duplicate zs: ' + str(count_duplicatez))
        print('Total removed due to bad slopes: ' + str(count_rem_slope))
        print('Total removed small tracks (<' + str(smallest_track_size) + ') hits: ' + str(count_small_tracks))

    return labels